# Training and Deploying the Fraud Detection Model

In this notebook, we will take the outputs from the Processing Job in the previous step and use it and train and deploy an XGBoost model. Our historic transaction dataset is initially comprised of data like timestamp, card number, and transaction amount and we enriched each transaction with features about that card number's recent history, including:

- `num_trans_last_10m`
- `num_trans_last_1w`
- `avg_amt_last_10m`
- `avg_amt_last_1w`

Individual card numbers may have radically different spending patterns, so we will want to use normalized ratio features to train our XGBoost model to detect fraud. 

### Imports 

In [ ]:
from sklearn.model_selection import train_test_split
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker import image_uris
import pandas as pd
import numpy as np
import sagemaker
import boto3
import io

### Essentials 

In [ ]:
LOCAL_DIR = './data'
BUCKET = sagemaker.Session().default_bucket()
PREFIX = 'training'

sagemaker_role = sagemaker.get_execution_role()
s3_client = boto3.Session().client('s3')

First, let's load the results of the SageMaker Processing Job ran in the previous step into a Pandas dataframe. 

In [ ]:
df = pd.read_csv(f'{LOCAL_DIR}/aggregated/processing_output.csv')
#df.dropna(inplace=True)
df['cc_num'] = df['cc_num'].astype(np.int64)
df['fraud_label'] = df['fraud_label'].astype(np.int64)
df.head()
len(df)

### Split DataFrame into Train & Test Sets

The artifically generated dataset contains transactions from `2020-01-01` to `2020-06-01`. We will create a training and validation set out of transactions from `2020-01-15` and `2020-05-15`, discarding the first two weeks in order for our aggregated features to have built up sufficient history for cards and leaving the last two weeks as a holdout test set. 

In [ ]:
training_start = '2020-01-15'
training_end = '2020-05-15'

training_df = df[(df.datetime > training_start) & (df.datetime < training_end)]
test_df = df[df.datetime >= training_end]

test_df.to_csv(f'{LOCAL_DIR}/test.csv', index=False)

Although we now have lots of information about each transaction in our training dataset, we don't want to pass everything as features to the XGBoost algorithm for training because some elements are not useful for detecting fraud or creating a performant model:
- A transaction ID and timestamp is unique to the transaction and never seen again. 
- A card number, if included in the feature set at all, should be a categorical variable. But we don't want our model to learn that specific card numbers are associated with fraud as this might lead to our system blocking genuine behaviour. Instead we should only have the model learn to detect shifting patterns in a card's spending history. 
- Individual card numbers may have radically different spending patterns, so we will want to use normalized ratio features to train our XGBoost model to detect fraud. 

Given all of the above, we drop all columns except for the normalised ratio features and transaction amount from our training dataset.

In [ ]:
training_df.drop(['tid','datetime','cc_num','num_trans_last_10m', 'avg_amt_last_10m',
       'num_trans_last_1w', 'avg_amt_last_1w'], axis=1, inplace=True)

The [built-in XGBoost algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) requires the label to be the first column in the training data:

In [ ]:
training_df = training_df[['fraud_label', 'amount', 'amt_ratio1','amt_ratio2','count_ratio']]
training_df.head()

In [ ]:
train, val = train_test_split(training_df, test_size=0.3)
train.to_csv(f'{LOCAL_DIR}/train.csv', header=False, index=False)
val.to_csv(f'{LOCAL_DIR}/val.csv', header=False, index=False)

In [ ]:
!aws s3 cp {LOCAL_DIR}/train.csv s3://{BUCKET}/{PREFIX}/
!aws s3 cp {LOCAL_DIR}/val.csv s3://{BUCKET}/{PREFIX}/

In [ ]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":"100"}

output_path = 's3://{}/{}/output'.format(BUCKET, PREFIX)

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", sagemaker.Session().boto_region_name, "1.2-1")

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path)

# define the data type and paths to the training and validation datasets
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}".format(BUCKET, PREFIX, 'train.csv'), content_type=content_type)
validation_input = TrainingInput("s3://{}/{}/{}".format(BUCKET, PREFIX, 'val.csv'), content_type=content_type)

# execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})

Ideally we would perform hyperparameter tuning before deployment, but for the purposes of this example will deploy the model that resulted from the Training Job directly to a SageMaker hosted endpoint.

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium',
    serializer=sagemaker.serializers.CSVSerializer(), wait=True)

In [ ]:
endpoint_name=predictor.endpoint_name
#Store the endpoint name for later cleanup 
%store endpoint_name
endpoint_name

Now to check that our endpoint is working, let's call it directly with a record from our test hold-out set. 

In [ ]:
payload_df = test_df.drop(['tid','datetime','cc_num','fraud_label','num_trans_last_10m', 'avg_amt_last_10m',
       'num_trans_last_1w', 'avg_amt_last_1w'], axis=1)
payload = payload_df.head(1).to_csv(index=False, header=False).strip()
payload

In [ ]:
float(predictor.predict(payload).decode('utf-8'))

## Show that the model predicts FRAUD / NOT FRAUD

In [ ]:
count_ratio = 0.30
payload = f'1.00,1.0,1.0,{count_ratio:.2f}'
is_fraud = float(predictor.predict(payload).decode('utf-8'))
print(f'With transaction count ratio of: {count_ratio:.2f}, fraud score: {is_fraud:.3f}')

In [ ]:
count_ratio = 0.06
payload = f'1.00,1.0,1.0,{count_ratio:.2f}'
is_fraud = float(predictor.predict(payload).decode('utf-8'))
print(f'With transaction count ratio of: {count_ratio:.2f}, fraud score: {is_fraud:.3f}')